<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/pull_one_hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz



In [303]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()

#filter = {
       # "MlsStatus": "SLD",
       # "ModificationTimestamp": {"$gte": dt_string}
        #}
#result = rets_client.search(resource='Property', resource_class='Property', limit = 10,search_filter=filter)





True

In [342]:
#Relist - Back on Market

tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 2)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")

relist_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(PriceChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
df = pd.DataFrame()
for item in relist_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence'):
      df = df.append(item, ignore_index=True)
df = pd.DataFrame(df)
df['地址'] = df['StreetNumber'] + " " + df['StreetName'] + " DR," + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']
df = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','地址']]
df.to_csv('BOM.csv')

In [344]:
#low_price
lowerPrice_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(PriceChangeTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))
lis = []
di ={}
df = pd.DataFrame()

for item in lowerPrice_result:
  if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousListPrice'] > item['ListPrice']):
    df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True) 
    di = item 
for key in di:
  lis.append(key)
df = df.T
df.columns = lis

pre_lis = []
aft_lis = []

df.dropna(subset=['PreviousListPrice'], inplace=True)


for item in df['ListPrice']:
  item = int(round(float(item),0))
  aft_lis.append(item)

for item in df['PreviousListPrice']:
  item = int(round(float(item),0))
  pre_lis.append(item)
  #print(type(item))

df['PreviousListPrice'] = pre_lis
df['ListPrice'] = aft_lis
df['percent'] = (df['PreviousListPrice'] - df['ListPrice'])/ df['ListPrice']

df['地址'] = df['StreetNumber'] + " " + df['StreetName'] + " DR," + df['City'] + ", " +df['StateOrProvince'] + " " + df['PostalCode']

df = df[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt','地址','percent']]
#df.to_csv('low_price.csv')
print(len(df.index))





11
